In [ ]:
######################## REQUIRED PACKAGES #############################
import cobra
from cobra.flux_analysis import flux_variability_analysis

import pandas as pd
import numpy as np
import seaborn as sb

import pickle
import scipy
import zlib
import random

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from Bio.KEGG import REST
from difflib import SequenceMatcher
from math import log
from collections import Counter

import localpytfa
import localpytfa.io
from localpytfa.io import load_thermoDB
from localpytfa.io.json import save_json_model, load_json_model
from localpytfa.io import import_matlab_model
from localpytfa.analysis import variability_analysis

In [ ]:
######################## MODEL INSPECTION #############################

def string_reaction(r):
    """Returns the chemical reaction as an easily readble string"""
    met_dict = {}
    met_dict_name = {}
    for m in r.metabolites:
        coeff = r.get_coefficient(m.id)
        met_dict_name[m.name] = coeff
    substrates_names = dict(filter(lambda x: x[1] < 0.0, met_dict_name.items())) 
    products_names = dict(filter(lambda x: x[1] > 0.0, met_dict_name.items())) 
    subs = [' '.join([str(substrates_names[key]),str(key)]) for key in substrates_names.keys()]
    subs = ' + '.join(subs)
    prods = [' '.join([str(products_names[key]),str(key)]) for key in products_names.keys()]
    prods = ' + '.join(prods)
    if r.lower_bound < 0.0:
        if r.upper_bound == 0.0:
            reaction_string = "{} <-- {} {}".format(subs,prods,r.compartments)
        else:
            reaction_string = "{} <--> {} {}".format(subs,prods,r.compartments)
    else: 
        reaction_string = "{} --> {} {}".format(subs,prods,r.compartments)
    #print(reaction_string)
    return(reaction_string)

In [ ]:
######################## EXO-METABOLITE CONSTRAINTS #############################

def get_concentrations(data,condition,time):
    concentration = list(data[met][(data['group'] == condition) & (data['time'] == time)])
    concentration = [float(i) for i in concentration]
    return(concentration)

def exchange_rate(i0,i,mu,time,b0=2.5e6,DW=2.5e-10):
    """Calculated the specific exchange rate.
    i0: initial concentration.
    i: concentration at time in exponential growth.
    mu: growth rate.
    b0: initial biomass."""
    q = ((i-i0)*mu*DW)/((np.exp(mu*DW*time)-1)*b0*DW)
    return(q)

def exchange_rate_adj(i0,i,mu,time,k=0.002,b0=2.5e6,DW=2.5e-10):
    """Adjusts for the decay of unstable chemicals."""
    q = ((i - (i0*np.exp(-k*time)))*(k+mu))/((np.exp(mu*time)-np.exp(-k*time))*b0)
    return(q)

In [ ]:
######################## THERMODYNAMIC CONSTRAINTS #############################

def apply_met_const(tmodel,dic,condition):
    """Add metabolite constraints to the model from the provided dictionary and conditions"""
    not_found = []
    met_const = []
    met_names = [m.name for m in tmodel.metabolites]
    scaling = max(get_all_values(dic))
    print("Scaling of {} applied to metabolite concentrations".format(scaling))
    for met_name in dic.keys():
        met_index = met_names.index(met_name)
        met = tmodel.metabolites[met_index]
        bounds = dic[met_name][condition]
        if try_apply_bounds(tmodel,met,bounds,scal=scaling):
            met_const.append(met.id)
        else:
            not_found.append(met.id)
    print("{} metabolite IDs not found in the thermodynamic model.".format(len(not_found)))
    print("{} metabolite constraints added to the thermodynamic model.".format(len(met_const)))
    
def try_apply_bounds(tmodel,met,bounds,scal):
    if 'nan' not in [str(i) for i in bounds]:
        try:
            the_conc_var = tmodel.log_concentration.get_by_id(met.id)
            the_conc_var.ub = log(bounds[0]/scal)
            the_conc_var.lb = log(bounds[1]/scal)
            return True
        except KeyError:
            return False
    else:
        return False
    
def create_thermo_model(dat,model):
    # Set up the thermodynamic model
    tmodel = localpytfa.ThermoModel(dat, model)
    tmodel.solver = 'gurobi'
    # TFA conversion
    tmodel.prepare()
    tmodel.convert(add_displacement = True) # not sure what the add_displacement does 
    # Info on the model
    tmodel.print_info()
    return(tmodel)
    
def add_pfba_constraint(tmodel,ret=False):
    pfba_solution = cobra.flux_analysis.pfba(tmodel)
    # Add the total sum of fluxes as a constraint to the model 
    coefficients = dict()
    for rxn in tmodel.reactions:
        coefficients[rxn.forward_variable] = 1.
        coefficients[rxn.reverse_variable] = 1.
    constraint = tmodel.problem.Constraint(0, lb=0, ub=pfba_solution.objective_value)
    tmodel.add_cons_vars(constraint)
    tmodel.solver.update()
    constraint.set_linear_coefficients(coefficients=coefficients)
    if ret:
        return(constraint) #pfba_solution.objective_value
    
def tfva(tmodel,condition):
    tmodel.optimize()
    tmodel.solver.update()
    tva_fluxes = variability_analysis(tmodel)
    pickle.dump(tva_fluxes,open("../Results/pTVA_{}.p".format(condition),"wb"))
    return(tva_fluxes)

def met_conc_set_default(model,lower=10**(-9),upper=0.1):
    """Set the default uppper and lower bounds for metabolite concentrations"""
    for conc_var in model.log_concentration:
        conc_var.variable.lb = log(lower)
        conc_var.variable.ub = log(upper)
    return(model)

def apply_solver_settings(model, solver):
    model.solver = solver
    # model.solver.configuration.verbosity = 1
    model.solver.configuration.tolerances.feasibility = 1e-9
    if solver == 'optlang_gurobi':
        model.solver.problem.Params.NumericFocus = 3
    model.solver.configuration.presolve = True
    
def get_all_values(d):
    """Get all nested values from a dictionary"""
    if isinstance(d, dict):
        for v in d.values():
            yield from get_all_values(v)
    elif isinstance(d, list):
        for v in d:
            yield from get_all_values(v)
    else:
        yield d 

In [ ]:
######################## THERMODYNAMIC ANNOTATIONS #############################

def annotate_model_recon(model):
    for i in [2,3]:
        recon_model = import_matlab_model('../Models/GEM_Recon{}_thermocurated_redHUMAN.mat'.format(i))
        recon_ids = set([m.name for m in recon_model.metabolites])
        human1_ids = set([m.name for m in model.metabolites])
        intersect = human1_ids.intersection(recon_ids)
        print(len(intersect))
        for m in list(intersect)[0:10]:
            rec_annot = [met.annotation for met in recon_model.metabolites if met.name == m][0]
            mets = [met for met in model.metabolites if met.name == m]
            for met in mets:
                met.annotation = rec_annot

def annotate_model_kegg(model,disp=False):
    """Annotates model metabolites using SEED ids based on their chemical formula."""
    counter = 0
    for m in model.metabolites:
        counter += 1 
        if disp is True:
            if (counter % 1000) == 0:
                print("{} out of {} metabolites checked".format(counter,len(model.metabolites)))
        if m.name == "O2" or m.name =="O2-":
            # ConnectionResetError: [Errno 104] Connection reset by peer
            m.annotation = {'seed_id': 'cpd00007'}
        else:
            # Find KEGG IDs that correspond to the chemical formula of the compound 
            try:
                if len(m.formula) > 0:
                    cpd = REST.kegg_find("compound", m.formula, "formula").read()
                    cpd = cpd.split()
                    if len(cpd)>0:
                        # If more than one corresponding KEGG ID is found iterate through all of them
                        ci_scores = {}
                        for ci in cpd:
                            # Check which name is closest to the one in the model and select that KEGG ID
                            if "cpd" in ci:
                                check_name = REST.kegg_get(ci).read()
                                check_name = check_name.split()
                                check_name = check_name[check_name.index("NAME")+1]
                                s_score = similar(m.name,check_name)
                                ci_scores[s_score] = ci
                        select_ci = ci_scores[max(ci_scores.keys())]
                        m_ann = 'cpd{}'.format(select_ci[5:])
                        m.annotation = {'seed_id':m_ann}
                    else:
                        m.annotation = {'seed_id':'NA'}
                else:
                    m.annotation = {'seed_id':'NA'}
            except: 
                print(m.id)
                print(m.name)
                print(m.formula)
                m.annotation = {'seed_id':'NA'}
                
def copy_compartments_recon(model,name_length=11):
    """Copies the compartment data from the Recon2 or Recon3 model"""
    recon2_model = import_matlab_model('../Models/GEM_Recon2_thermocurated_redHUMAN.mat')
    recon3_model = import_matlab_model('../Models/GEM_Recon3_thermocurated_redHUMAN.mat')
    # Add a compartment for metabolites without one 
    comp_dict = {'c':'c','m':'m','p':'x','s':'e','x':'e','r':'r','l':'l','n':'n','g':'g','i':'i'}
    for m in model.metabolites:
        if m.compartment not in comp_dict.keys():
            loc = 3 + name_length
            m.compartment = m.id[-loc]
    # Copy over the compartments from the Recon Models 
    for recon_model in [recon2_model,recon3_model]:
        # Compartments
        comp_names = np.array(list(model.compartments.keys()))
        comp_names = np.append(comp_names,'n')
        comp_names = np.append(comp_names,'i')
        new_comp_dict = {}
        for comp in comp_names:
            recon_comp = comp_dict[comp]
            try:
                info = recon_model.compartments[recon_comp]
                new_comp_dict[comp] = info
            except KeyError:
                pass
        model.compartments = new_comp_dict
    # Check for how many of the annotated metabolites, thermodynamics data is available
    with open('../Data/thermo_data.thermodb', 'rb') as file:
        ReactionDB = pickle.loads(zlib.decompress(file.read()))
    check = set(ReactionDB['metabolites'].keys())
    print("{} of the annotated metabolites are in the database.".format(len(check_annot(model,check))))
    
def similar(a, b):
    """Returns a similarity score for two strings"""
    return SequenceMatcher(None, a, b).ratio()

def check_annot(model,thermo):
    """Check how many model annotations are present in the thermodynamic database"""
    ann = []
    for m in model.metabolites:
        try:
            ann.append(m.annotation['seed_id'])
        except KeyError:
            pass
    return(set(ann).intersection(thermo))

In [ ]:
######################## FLUX ANALYSIS #############################

def normalize_growth(tva,tmodel):
    bm_tva = float(tva.loc["HCC_biomass"]["minimum"])
    for r in tmodel.reactions: 
        for m in ["minimum","maximum"]:
            tva.at[r.id,m] = float(tva.loc[r.id][m])/bm_tva
            
sub_manual = {'Pyruvate Metabolism':['Pyruvate'], 
              'Pentose Phosphate Pathway':['Pentose'],
              'Oxidative Phosphorylation':['Oxidative'],
              'Glycolysis / Gluconeogenesis':['Glycolysis'],
              'Tricarboxylic acid cycle':['Butanoate','Tricarboxylic'],
              'Fatty Acid Metabolism':['atty acid','Carnitine','Propanoate','Arachidonic','Acyl-CoA','Butanoate','Linoleate','Glycero','Acylglycerides','C5-branched'],
              'Beta oxidation':['Beta oxidation'],
              'Cholesterol Metabolism':['holesterol','Terpenoid'],
              'Lipid Metabolism':['lipid','Eicosanoid','Prostaglandin','Leukotriene'],
              'Bile Acid Metabolism':['Bile acid'],
              'Nucleotide metabolism':['ucleotide','Pyrimidine','Purine'],
              'ROS detoxification':['ROS','Glutathione'],
              'Vitamin Metabolism':['Vitamin','Retinol','Thiamine','Riboflavin','Biotin','Folate','nicotinamide','Ascorbate','Pantothenate'],
              'Starch and Sugar Metabolism':['Starch','Fructose','Galactose','Pentose','Inositol'],
              'Amino Acid Metabolism':['amino acids','Lysine','Histidine','Cysteine','Protein','lanine','yrosine','Glycine',
                                       'Arginine','Valine','Tryptophan','Nitrogen'],
              'Hormone Metabolism':['Serotonin','Estrogen','Androgen','Steroid','Glucocorticoid'],
              'Exchange Reactions':['Exchange'],
              'Transport Reactions':['Transport'],
              'Other':['["]','Pool','Drug','Artificial','Blood','Porphyrin','Miscellaneous',
                       'Heme','xenobiotics','Aminoacyl','Biopterin','Other'],
              'Sulfur Metabolism':['Sulfur','Chondroitin','sulfate'],
              'Glycosylation':['glycan','Glycosyl']}

def diff_scale_flux_subsystem(tva_sensitive,tva_resistant,subsystems,p_diff=0.15):
    # Plot resistant against sensitive
    lb_sensitive = []
    ub_sensitive = []
    lb_resistant = []
    ub_resistant = []
    names = []
    rxns = []
    percent_differences = []
    for r in tmodel.reactions:
        rxn_subsystem = r.subsystem
        sub_find = [rxn_subsystem.find(s) for s in subsystems]
        if len([i for i in range(len(sub_find)) if sub_find[i]!=-1]) > 0:
            # Check if lower and upper bounds are different
            lb1 = tva_sensitive.loc[r.id]['minimum']
            ub1 = tva_sensitive.loc[r.id]['maximum']
            lb2 = tva_resistant.loc[r.id]['minimum']
            ub2 = tva_resistant.loc[r.id]['maximum']
            percent_difference = percent_diff_fluxes(lb1,ub1,lb2,ub2,p_diff)
            if percent_difference != False:
                met_name = [m.name for m in list(r.metabolites.keys())][0]
                lb_sensitive.append(lb1)
                ub_sensitive.append(ub1)
                lb_resistant.append(lb2)
                ub_resistant.append(ub2)
                names.append(met_name)
                percent_differences.append(percent_difference)
                rxns.append(r)
    return lb_sensitive, ub_sensitive, lb_resistant, ub_resistant, names, rxns, percent_differences

def percent_diff(x1,x2,zero_cut):
    max_val = max([abs(x1),abs(x2)])
    if max_val > zero_cut:
        diff = abs((x1-x2))/max_val
        return(diff)
    else:
        return(0.0)
    
def percent_diff_fluxes(lower1,upper1,lower2,upper2,p_diff = 0.15,zero_cut=0.001):
    """Checks whether both the lower and upper bounds of two reactions
    differ by a given percentage."""
    diff_lower = abs(lower1-lower2)
    diff_upper = abs(upper1-upper2)
    if diff_lower > zero_cut and diff_upper > zero_cut:
        p_diff_lower = percent_diff(lower1,lower2,zero_cut)
        p_diff_upper = percent_diff(upper1,upper2,zero_cut)
        if p_diff_lower > p_diff and p_diff_upper > p_diff:
            min_diff = min([p_diff_lower,p_diff_upper])
            return(min_diff)
    return(False) 

def feature_scaling(values,lb,ub):
    """Rescaling (min-max) normalization"""
    scaled_values = [(x-lb)/(ub-lb) for x in values]
    return(scaled_values)

def compare_fluxes(flux1_lb,flux1_ub,flux2_lb,flux2_ub,percent_difference=0.01,zero_cut=0.001): 
    """Returns all reactions that have a diff. ub & lb"""
    # Compare the two flux solutions
    altered_reactions = []
    for i in range(len(flux1_lb)):
        diff_lower = abs(flux1_lb[i]-flux2_lb[i])
        diff_upper = abs(flux1_ub[i]-flux2_ub[i])
        if diff_lower > zero_cut and diff_upper > zero_cut:
            if diff_lower > percent_difference*max([abs(flux1_lb[i]),abs(flux2_lb[i])]):
                if diff_upper > percent_difference*max([abs(flux1_ub[i]),abs(flux2_ub[i])]):
                    altered_reactions.append(i)
    print("FVA: {} reactions have a different upper and lower bound.".format(len(altered_reactions)))
    return(altered_reactions)

def relative_flux(condition_value,control_value):
    max_val = max([control_value,condition_value])
    if max_val != 0.:
        relative_diff = (condition_value - control_value)/control_value #max_val
        return(relative_diff) #negative to reverse the colorbar
    else:
        return(0.)

In [ ]:
######################## SIMULATE HYPOXIA #############################

def plot_o2_pe(reac_id,o2_fluxes,growth_norm=True,pfba_sum=False):
    fs = 16
    plt.figure(figsize=(6,5))
    cols = ["blue","red","black","orange"]
    labs = ["WT","OxR","WT-DMSO","RuR"]
    all_flux = []
    for i,model in enumerate(models):
        r = model.reactions.get_by_id(reac_id)
        fva_sols_min = []
        fva_sols_max = []
        pfba_sols = []
        bm_fluxes = []
        for constraint in o2_fluxes:
            ex_o2 = model.reactions.get_by_id("HMR_9048")
            ex_o2.lower_bound = constraint
            ex_o2.upper_bound = constraint
            try:
                bm = model.reactions.get_by_id("HCC_biomass")
                bm_pfba = cobra.flux_analysis.pfba(model,reactions=[bm])
                bm_pfba = bm_pfba[bm.id]
            except:
                bm_pfba = np.nan
            try:
                if pfba_sum:
                    new_model = model.copy()
                    pfba_sum = add_pfba_constraint(new_model,ret=True)
                fva = flux_variability_analysis(model,r)
                pfba = cobra.flux_analysis.pfba(model,reactions=r)
                fva_sols_min.append(float(fva['minimum']))
                fva_sols_max.append(float(fva['maximum']))
                pfba_sols.append(float(pfba[r.id]))
            except:
                fva_sols_min.append(np.nan)
                fva_sols_max.append(np.nan)
                pfba_sols.append(np.nan)
            bm_fluxes.append(bm_pfba)
        rxn_fluxes_min = fva_sols_min
        rxn_fluxes_max = fva_sols_max
        # Normalize fluxes according to biomass 
        if growth_norm:
            rxn_fluxes_min = [v/b for v,b in zip(rxn_fluxes_min,bm_fluxes)]
            rxn_fluxes_max = [v/b for v,b in zip(rxn_fluxes_max,bm_fluxes)]
            o2_fluxes_plot = [v/b for v,b in zip(o2_fluxes,bm_fluxes)]
        else:
            o2_fluxes_plot = o2_fluxes
        all_flux = all_flux + rxn_fluxes_min + rxn_fluxes_max
        plt.plot(o2_fluxes_plot,pfba_sols,color=cols[i],label=labs[i],alpha=0.5,linewidth=3)
#         plt.plot(o2_fluxes_plot,rxn_fluxes_min,color=cols[i],label=None,linestyle='dashed')
#         plt.plot(o2_fluxes_plot,rxn_fluxes_max,color=cols[i],label=None,linestyle='dashed')
    plt.xlabel("Oxygen Flux",fontsize=fs)
    #plt.ylabel("{}".format(reac_id),fontsize=fs)
    plt.ylabel("{}".format(string_reaction(r)),fontsize=fs)
    plt.xticks(fontsize=fs)
    plt.yticks(fontsize=fs)
    #plt.ylim(min(all_flux)-abs(min(all_flux)*0.05),max(all_flux)+abs(min(all_flux)*0.05))
    print(string_reaction(r))
    print(r.reaction)
    print(reac_id)
    plt.legend(fontsize=fs)

In [ ]:
######################## ANNOTATE SUBSYSTEMS #############################

def subsystem_transfer(model):
    # Transfer the subsystems from the Robinson et al. 2020 model 
    HCT_model = cobra.io.load_matlab_model("../Models/HCT_GEM.mat")
    HCT_rxns = set([r.id for r in HCT_model.reactions])
    model_rxns = set([r.id for r in model.reactions])
    # Can do a direct transfer for most of the reactions
    for rid in model_rxns.intersection(HCT_rxns):
        subsystem = HCT_model.reactions.get_by_id(rid).subsystem
        r_model = model.reactions.get_by_id(rid)
        r_model.subsystem = subsystem 
    # Where a direct transfer is not possible use the metabolites IDs instead
    for rid in model_rxns.difference(HCT_rxns):
        r_model = model.reactions.get_by_id(rid)
        if len(r_model.products) == 0. or len(r_model.reactants) == 0.:
            r_model.subsystem = "Exchange"
        elif [x.id[:-1] for x in r_model.products] == [x.id[:-1] for x in r_model.reactants]:
            r_model.subsystem = "Transport"
        elif "pool" in string_reaction(r_model):
            r_model.subsystem = "Pool"
        else: 
            # Check which subsystem the metabolites of that reactions belong to
            met_subs = []
            for m in r_model.metabolites:
                if m.name in ["H2O","O2","H+","O2-","Pi","NADH","NADPH","NAD+","NADP+"]:
                    pass
                else:
                    try:
                        m_HCT = HCT_model.metabolites.get_by_id(m.id)
                        for r_HCT in m_HCT.reactions:
                            met_subs.append(r_HCT.subsystem)
                    except KeyError:
                        pass
            met_subs = [s for s in met_subs if 'Transport' not in s]
            if len(met_subs) == 0:
                met_subs = ["Other"]
            # Select the most commonly occuring subsystem for those metabolites 
            select_sub = max(set(met_subs), key = met_subs.count)
            r_model.subsystem = select_sub
            
            
def clean_sub_name(sub_name):
    sub_name = sub_name.replace("[array(['","")
    sub_name = sub_name.split("'")
    if len(sub_name) > 1:
        sub_name = sub_name[0]
    return(sub_name)